# А/В тесты

<p><img src="https://www.invespcro.com/blog/images/blog-images/ab-test-1-1.jpg" alt="" style="width: 400px;"></p>

In [1]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportion_confint

## Задача 1 

На сайте выставляется баннер с рекламой. Хочется узнать, насколько он нравится пользователям. Для того, чтобы это сделать, нужно померить, как часто пользователи кликают на этот баннер. <br>
Чтобы оценить долю кликов честно, нужно подождать существенное время. Но хочется понять на начальном этапе, что баннер совсем неудачный и его нужно убрать. В таком случае мы можем оценить долю успеха этого баннера. <br> <br>
В табличке `banner_results.csv` содержатся результаты по кликам.
 * Найдите долю кликов. Будем обозначать ее как $\hat p$
 * Стандартное отклонение для доли считается по формуле $ \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}}.$ Найдите его.
 * Можно показать, что $\hat p$ имеет нормальное распределение. Постройте на основе правила двух сигм для вашей оценки доли доверительный интервал. Найдите его ширину.

Строим доверительный интервал для $p$ по формуле:

$$\hat{p}\pm 1.96 \cdot \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [2]:
df = pd.read_csv('banner_results.csv')

In [3]:
df.head()

,banner
0,1
1,1
2,1
3,0
4,1


In [4]:
df['banner'].value_counts()

1    502
0    498
Name: banner, dtype: int64

In [5]:
results = df['banner'].values

___

In [6]:
n = len(results) # количество наблюдений
m = sum(results) # количество успехов
p_hat = m / n # доля успехов

std = np.sqrt(p_hat * (1 - p_hat) / n)

# границы доверительного интервала
left =  p_hat - 1.96 * std
right = p_hat + 1.96 * std

print('Доверительный интервал: [{:.3f}, {:.3f}]'.format(left,right))

Доверительный интервал: [0.471, 0.533]


In [7]:
normal_interval = proportion_confint(m, n, alpha = 0.05)
normal_interval

(0.4710104963037765, 0.5329895036962234)

In [8]:
print('Доверительный интервал: [{:.3f}, {:.3f}]'.format(normal_interval[0], normal_interval[1]))
print('Ширина интервала:', normal_interval[1] - normal_interval[0])

Доверительный интервал: [0.471, 0.533]
Ширина интервала: 0.06197900739244694


## Задача 2

У нас есть данные цен за квартиры в штате Аризона. Хотим сравнить их среднее значение цены со средней ценой во всей стране и сделать вывод, отличаются ли цены в среднем или нет.

In [9]:
df_prices = pd.read_csv('housing_prices.csv')

In [10]:
df_prices.head()

,price
0,273023.0
1,165992.0
2,124175.0
3,437547.0
4,336444.0


In [11]:
df_prices.mean()

price    275964.96
dtype: float64

### Проверка гипотезы
Мы хотим узнать, есть ли значительное отличие в средних значениях цен на жилье. Средняя цена на жилье во всей стране $\\mu=289100$. Для теста выбираем уровень значимости равный 5%. 

$H_0:$ $\bar{x} = \mu$ -- Нет значительной разницы в средних <br>
$H_a:$ $\bar{x} \neq \mu$ -- Есть значительная разица в средних

In [12]:
mu = 289100

In [13]:
n = df_prices['price'].count()
mean = df_prices['price'].mean()
std = df_prices['price'].std(ddof=1)
n, mean, std

(100, 275964.96, 111727.88368254455)

Строим доверительный интервал $$ 
\left((\mu - \bar x) - 1.96 \cdot \frac{\hat \sigma}{\sqrt{n}}; \quad (\mu - \bar x) + 1.96 \cdot \frac{\hat \sigma}{\sqrt{n}} \right)$$

In [14]:
mean - mu

-13135.039999999979

In [15]:
left = (mean - mu) - 1.96 * std / np.sqrt(n)
right = (mean - mu) + 1.96 * std / np.sqrt(n)

print('Доверительный интервал: [{:.2f}, {:.2f}]'.format(left,right))

Доверительный интервал: [-35033.71, 8763.63]


Теперь то же самое с помощью t-статистики $$
t = \frac{\mu_0 - \bar x}{\frac{\hat \sigma}{\sqrt{n}}}
$$

In [16]:
t_nabl = (mean - mu) / (std / np.sqrt(n))
print(t_nabl)

-1.1756277454713921


In [17]:
from statsmodels.stats.weightstats import _zconfint_generic

_zconfint_generic(mean - mu,  std / np.sqrt(n), 0.05, 'two-sided')

(-35033.30280866675, 8763.22280866679)